# Step 10: Travel Agent Assistant

This notebook demonstrates a specialized travel agent assistant with system instructions.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
import asyncio
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from plugins.datetime import DateTimePlugin
from plugins.location import LocationPlugin
from plugins.weather import WeatherPlugin

In [ ]:
# Initialize kernel and plugins
kernel = Kernel()
azure_chat_completion = AzureChatCompletion(service_id="travel_agent_chat")
kernel.add_service(azure_chat_completion)

kernel.add_plugin(LocationPlugin(), plugin_name="location_plugin")
kernel.add_plugin(DateTimePlugin(), plugin_name="time_plugin")
kernel.add_plugin(WeatherPlugin(), plugin_name="weather_plugin")

In [ ]:
# Configure automatic function calling
execution_settings = AzureChatPromptExecutionSettings(
    function_choice_behavior=FunctionChoiceBehavior.Auto(
        auto_invoke=True,
        filters={
            "included_plugins": [
                "time_plugin",
                "location_plugin",
                "weather_plugin",
            ]
        },
    )
)

In [ ]:
# Initialize chat with system message
history = ChatHistory()
history.add_system_message(
    "You are travel agent expert, Tripy, be polite and introduce yourself after first user message and make sure you are helpful and give only trip advice, if there is anything else, do not respond and say you don't know"
)

In [ ]:
# Interactive chat function
async def chat(user_input: str):
    history.add_user_message(user_input)
    result = await azure_chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )
    print("Assistant > " + str(result))
    history.add_message(result)
    return result

In [ ]:
# Example: Start conversation
await chat("Hello!")

In [ ]:
# Example: Ask for travel advice
await chat("What should I visit today?")

In [ ]:
# Interactive chat loop
# Type 'exit' to quit
while True:
    user_input = input("User > ")
    
    if user_input == "exit":
        break
    
    await chat(user_input)